In [2]:
import numpy as np
import time

class DualGradientMethod:
    def __init__(self, A, b, penalty, gamma_u, gamma_d, L_0, v_0, max_iter=10000, tol=1e-6) -> None:
        self.A = A
        self.b = b
        self.penalty = penalty
        self.gamma_u = gamma_u
        self.gamma_d = gamma_d
        self.L = L_0
        self.v = v_0
        self.max_iter = max_iter
        self.tol = tol
        self.gap_history = []
        self.loss_history = []
        self.cpu_time_history = []

    def gradient_f(self, x):
        return 2 * np.dot(self.A.T, (np.dot(self.A, x) - self.b))

    def f(self, x):
        return np.linalg.norm(np.dot(self.A, x) - self.b)**2 / 2

    def psi(self, x):
        # Placeholder for potential regularizer function
        return 0

    def compute_steps(self):
        v_prev = self.v
        initial_residual = np.dot(self.A, self.v) - self.b
        initial_loss = np.linalg.norm(initial_residual)**2 / 2
        
        for k in range(self.max_iter):
            start_time = time.process_time()

            # Nesterov acceleration
            if k > 0:
                y = self.v + (k - 1) / (k + 2) * (self.v - v_prev)
            else:
                y = self.v

            grad_f = self.gradient_f(y)
            Mk = np.linalg.norm(grad_f, 2)
            if Mk == 0:
                Mk = 1e-16  # Prevent division by zero

            L_k = max(self.L, Mk / self.gamma_d)
            T, L_new = self.gradient_iteration(self.penalty, self.gamma_u, y, L_k)
            self.L = L_new
            
            v_prev = self.v
            self.v = T
            
            current_residual = np.dot(self.A, self.v) - self.b
            current_loss = np.linalg.norm(current_residual)**2 / 2
            current_gap = current_loss / initial_loss

            end_time = time.process_time()

            self.gap_history.append(current_gap)
            self.loss_history.append(current_loss)
            self.cpu_time_history.append(end_time - start_time)

            if current_gap < self.tol:
                break

        return self.v, k, current_gap, self.gap_history, self.loss_history, self.cpu_time_history

    def gradient_iteration(self, penalty, gamma_u, x, M):
        L = M
        while True:
            changes = self.gradient_f(x) - L * x
            T = three_cases(changes, penalty, L)
            if not self.psi(T) > np.dot(self.gradient_f(T), (x - T)) + np.linalg.norm(x - T)**2 * L / 2 + self.psi(x):
                break
            L *= gamma_u
        return T, L

def three_cases(changes, penalty, denominator):
    # Vectorized handling of three cases for the proximal gradient update
    return (np.less_equal(changes, -penalty) * (-changes - penalty) + np.greater_equal(changes, penalty) * (-changes + penalty)) / denominator



In [6]:
import numpy as np
import scipy.sparse as sp

def generate_sparse_least_squares(m, n, rho):
    """
    Generates a Sparse Least Squares problem according to the specifications in Nesterov's paper.

    Parameters:
    m (int): Number of rows in matrix A.
    n (int): Number of columns in matrix A, n > m.
    rho (float): Sparsity and magnitude control parameter.

    Returns:
    A (numpy.ndarray): Generated dense matrix A of shape (m, n).
    b (numpy.ndarray): Generated vector b of shape (m,).
    x_star (numpy.ndarray): Sparse solution x* of shape (n,).

    Example:
    m, n, rho = 10, 20, 0.1  # dimensions and sparsity level
    A, b, x_star = generate_sparse_least_squares(m, n, rho)

    # print("Matrix A:\n", A)
    # print("Vector b:\n", b)
    print("Sparse solution x*:\n", x_star)
    """
    # Generate a dense matrix A with elements uniformly distributed in [-1, 1]
    A = np.random.uniform(-1, 1, (m, n))

    # Generate a sparse solution x_star
    x_star = np.zeros(n)
    # Ensure sparsity in the solution
    non_zero_indices = np.random.choice(n, int(n * rho), replace=False)
    x_star[non_zero_indices] = np.random.normal(0, 1, int(n * rho))

    # Calculate the vector b = A*x_star + noise
    # noise = np.random.normal(0, 0.1, m)  # adding small Gaussian noise
    b = np.dot(A, x_star) # + noise

    return A, b, x_star

# Parameters
m, n = 1000, 4000  # dimensions
rho = 0.1          # sparsity factor
tol = 1e-9         # tolerance for relative gap reduction

# Generate the problem
A, b, x_star = generate_sparse_least_squares(m, n, rho)

# Instantiate and solve using DualGradientMethod
v0 = np.zeros(n)
L0 = 2
gamma_d = 2
penalty = 0.5  # This replaces lambda_ which was likely a regularization parameter

solver = DualGradientMethod(A, b, penalty, gamma_d=2, gamma_u=1.1, L_0=L0, v_0=v0, max_iter=10000, tol=tol)
solution_dual, iterations_dual, final_gap_dual, gap_history_dual, loss_history_dual, cpu_history_dual = solver.compute_steps()

# Analyzing gap magnitudes and CPU time
def find_magnitude_indices(gap_history, func=np.log10):
    magnitudes = np.floor(func(gap_history))
    unique_magnitudes = []
    indices = []
    for i, magnitude in enumerate(magnitudes):
        if magnitude not in unique_magnitudes:
            unique_magnitudes.append(magnitude)
            indices.append(i + 1)
    indices = np.array(indices)
    return indices, unique_magnitudes

indices_dual, magnitudes_dual = find_magnitude_indices(gap_history_dual, func=np.log2)
cpu_history_dual = np.array(cpu_history_dual).cumsum()[indices_dual - 1]  # Adjusting indices for Python's 0-based index

# Output results for analysis
print(f"Solution: {solution_dual[:5]}...")  # Showing first few elements of the solution
print(f"Iterations: {iterations_dual}")
print(f"Final Gap: {final_gap_dual}")
print(f"Indices of significant gap changes: {indices_dual}")
print(f"CPU time at significant changes: {cpu_history_dual}")


Solution: [-0.4449078  -0.12895728  0.          0.          0.        ]...
Iterations: 9999
Final Gap: 9.121210451221099e-07
Indices of significant gap changes: [   1    2    3    5    6    7    8    9   11   12   13   16   20   24
   26   29   37   40   42   65 1339]
CPU time at significant changes: [ 0.03125   0.09375   0.09375   0.15625   0.171875  0.234375  0.234375
  0.25      0.34375   0.375     0.40625   0.46875   0.609375  0.78125
  0.84375   0.921875  1.234375  1.296875  1.34375   2.125    25.625   ]


In [7]:
import numpy as np
import time

class PrimalGradientMethod:
    def __init__(self, A, b, x0, penalty, gamma_u, L_0, tol, max_iter=10000) -> None:
        self.A = A
        self.b = b
        self.x = x0
        self.penalty = penalty
        self.gamma_u = gamma_u
        self.gamma_d = 1 / gamma_u  # As gamma_d seems to be the inverse process in gradient_iteration
        self.L = L_0
        self.tol = tol
        self.max_iter = max_iter
        self.gap_history = []
        self.loss_history = []
        self.cpu_time_history = []

    def gradient_f(self, x):
        """Computes the gradient of the least squares function."""
        return 2 * np.dot(self.A.T, (np.dot(self.A, x) - self.b))

    def f(self, x):
        """Computes the loss (norm of residual squared)."""
        return np.linalg.norm(np.dot(self.A, x) - self.b)**2

    def compute_steps(self):
        initial_loss = self.f(self.x)
        self.loss_history.append(initial_loss)
        
        for k in range(self.max_iter):
            start_time = time.process_time()

            gradient = self.gradient_f(self.x)
            step_size = 0.1 / (np.linalg.norm(gradient) if np.linalg.norm(gradient) != 0 else 1e-16)
            self.x -= step_size * gradient

            current_loss = self.f(self.x)
            relative_gap = current_loss / initial_loss

            self.loss_history.append(current_loss)
            self.gap_history.append(relative_gap)
            end_time = time.process_time()
            self.cpu_time_history.append(end_time - start_time)

            if relative_gap < self.tol:
                break

        return self.x, k, relative_gap, self.gap_history, self.loss_history, self.cpu_time_history

# Example Usage
# Parameters
m, n = 1000, 4000  # dimensions
rho = 0.1          # sparsity factor
tol = 1e-9         # tolerance for relative gap reduction

# Generate the problem
A, b, x_star = generate_sparse_least_squares(m, n, rho)
x0 = np.zeros(n)
penalty = 0.5
gamma_u = 1.1
L_0 = 2
tol = 1e-6
max_iter = 10000

solver = PrimalGradientMethod(A, b, x0, penalty, gamma_u, L_0, tol, max_iter)
solution, iterations, final_gap, gap_history, loss_history, cpu_time = solver.compute_steps()

print(f"Solution: {solution[:5]}...")  # Showing first few elements of the solution
print(f"Iterations: {iterations}")
print(f"Final Gap: {final_gap}")

print(f"Solution: {solution_dual[:5]}...")  # Showing first few elements of the solution
print(f"Iterations: {iterations_dual}")
print(f"Final Gap: {final_gap_dual}")
print(f"Indices of significant gap changes: {indices_dual}")
print(f"CPU time at significant changes: {cpu_time}")


Solution: [-0.05443786 -0.18406338 -0.0289667   0.27226145  0.05750593]...
Iterations: 9999
Final Gap: 4.741904856153861e-05


In [8]:
import numpy as np
import scipy.sparse as sp
import time

def generate_sparse_least_squares(m, n, rho):
    """
    Generate a sparse least squares problem where `A` is an m x n matrix,
    `b` is an m-dimensional vector, and `x_star` is a sparse n-dimensional vector.
    """
    A = np.random.randn(m, n)
    x_star = sp.rand(n, 1, density=rho).toarray().ravel()  # n-dimensional, sparsity defined by rho
    b = np.dot(A, x_star) + np.random.randn(m) * 0.1  # Adding some noise
    return A, b, x_star

class PrimalGradientMethod:
    def __init__(self, A, b, x0, tol, max_iter=10000):
        self.A = A
        self.b = b
        self.x = x0
        self.tol = tol
        self.max_iter = max_iter
        self.gap_history = []
        self.loss_history = []
        self.cpu_time_history = []

    def gradient_f(self, x):
        return 2 * np.dot(self.A.T, (np.dot(self.A, x) - self.b))

    def f(self, x):
        return np.linalg.norm(np.dot(self.A, x) - self.b)**2

    def compute_steps(self):
        initial_loss = self.f(self.x)
        self.loss_history.append(initial_loss)
        
        for k in range(self.max_iter):
            start_time = time.process_time()
            gradient = self.gradient_f(self.x)
            step_size = 0.1 / (np.linalg.norm(gradient) if np.linalg.norm(gradient) != 0 else 1e-16)
            self.x -= step_size * gradient
            current_loss = self.f(self.x)
            relative_gap = current_loss / initial_loss

            self.loss_history.append(current_loss)
            self.gap_history.append(relative_gap)
            end_time = time.process_time()
            self.cpu_time_history.append(end_time - start_time)

            if relative_gap < self.tol:
                break

        return self.x, k, relative_gap, self.gap_history, self.loss_history, self.cpu_time_history

# Parameters
m, n = 1000, 4000  # dimensions
rho = 0.1          # sparsity factor
tol = 1e-9         # tolerance for relative gap reduction

# Generate the problem
A, b, x_star = generate_sparse_least_squares(m, n, rho)
x0 = np.zeros(n)

# Instantiate and solve using PrimalGradientMethod
solver = PrimalGradientMethod(A, b, x0, tol, max_iter=10000)
solution, iterations, final_gap, gap_history, loss_history, cpu_time = solver.compute_steps()

# Analyzing gap magnitudes and CPU time
def find_magnitude_indices(gap_history, func=np.log2):
    magnitudes = np.floor(func(gap_history))
    unique_magnitudes = []
    indices = []
    for i, magnitude in enumerate(magnitudes):
        if magnitude not in unique_magnitudes:
            unique_magnitudes.append(magnitude)
            indices.append(i + 1)
    indices = np.array(indices)
    return indices, unique_magnitudes

indices, magnitudes = find_magnitude_indices(gap_history)
cpu_time = np.array(cpu_time).cumsum()[indices - 1]  # Adjusting indices for Python's 0-based index

# Output results for analysis
print(f"Solution: {solution[:5]}...")  # Showing first few elements of the solution
print(f"Iterations: {iterations}")
print(f"Final Gap: {final_gap}")
print(f"Indices of significant gap changes: {indices}")
print(f"CPU time at significant changes: {cpu_time}")


Solution: [ 0.17145659 -0.00623629 -0.04250588  0.16693029 -0.1020973 ]...
Iterations: 9999
Final Gap: 0.00027067967329087316
Indices of significant gap changes: [ 1 15 27 35 42 47 50 53 56 57 58 59 60 61 63 64]
CPU time at significant changes: [0.       0.109375 0.21875  0.3125   0.375    0.421875 0.421875 0.4375
 0.46875  0.46875  0.46875  0.484375 0.484375 0.484375 0.484375 0.484375]


In [11]:
import numpy as np
import time
from math import sqrt

def soft_thresholding(x, lambda_):
    """ Apply soft thresholding for LASSO. """
    return np.sign(x) * np.maximum(np.abs(x) - lambda_, 0.0)

def lasso_objective(X, y, beta, lambda_):
    """ Compute LASSO objective function. """
    n = len(y)
    residual = y - X.dot(beta)
    loss = 0.5 * np.dot(residual, residual) / n
    penalty = lambda_ * np.linalg.norm(beta, ord=1)
    return loss + penalty

def lasso_gradient(X, y, beta, lambda_):
    """ Compute gradient for LASSO. """
    n = len(y)
    residual = y - X.dot(beta)
    gradient_loss = -X.T.dot(residual) / n
    gradient_penalty = lambda_ * np.sign(beta)
    return gradient_loss + gradient_penalty

class AcceleratedLassoMethod:
    def __init__(self, X, y, lambda_, lr=0.01, max_iter=1000, tol=1e-6):
        self.X = X
        self.y = y
        self.lambda_ = lambda_
        self.lr = lr
        self.max_iter = max_iter
        self.tol = tol
        self.beta = np.zeros(X.shape[1])
        self.t = 1
        self.loss_history = []
        self.gap_history = []
        self.cpu_time_history = []

    def compute_steps(self):
        beta_prev = np.zeros_like(self.beta)

        for i in range(self.max_iter):
            start_time = time.process_time()
            t_prev = self.t
            self.t = 0.5 * (1 + sqrt(1 + 4 * t_prev**2))
            y_tilde = self.beta + ((t_prev - 1) / self.t) * (self.beta - beta_prev)
            gradient = lasso_gradient(self.X, self.y, y_tilde, self.lambda_)
            beta_prev = self.beta.copy()
            self.beta = soft_thresholding(y_tilde - self.lr * gradient, self.lr * self.lambda_)
            loss = lasso_objective(self.X, self.y, self.beta, self.lambda_)
            self.loss_history.append(loss)
            end_time = time.process_time()
            self.cpu_time_history.append(end_time - start_time)

            # Compute convergence gap and check termination condition
            gap = np.linalg.norm(self.beta - beta_prev) / (np.linalg.norm(beta_prev) if np.linalg.norm(beta_prev) != 0 else 1)
            self.gap_history.append(gap)
            if gap < self.tol:
                break

        return self.beta, self.loss_history, self.gap_history, self.cpu_time_history

# Example usage:
# Assuming X and y are your data matrix and target vector
lambda_ = 0.3
lr = 0.01
max_iter = 1000
tol = 1e-6
solver = AcceleratedLassoMethod(A, b, lambda_, lr, max_iter, tol)
solution, loss_history, gap_history, cpu_time_history = solver.compute_steps()

# Output results for analysis
print(f"Solution: {solution[:5]}...")  # Showing first few elements of the solution
print(f"Loss history: {loss_history[:5]}...")  # First few loss values
print(f"Gap history: {gap_history[:5]}...")  # First few gaps
print(f"CPU time history: {cpu_time_history[:5]}...")  # First few CPU times


print(f"Solution: {solution[:5]}...")  # Showing first few elements of the solution
print(f"Iterations: {iterations}")
print(f"Final Gap: {final_gap}")
print(f"Indices of significant gap changes: {indices}")
print(f"CPU time at significant changes: {cpu_time}")


Solution: [ 0.03043061  0.         -0.00111708  0.00376453 -0.0003996 ]...
Loss history: [60.64288288181856, 59.74762287190232, 58.70750855252287, 57.651181306609, 56.65804949290718]...
Gap history: [0.13086156823504916, 0.7044536615267238, 0.6306883361071357, 0.38739959263401974, 0.3220556553039628]...
CPU time history: [0.015625, 0.015625, 0.015625, 0.0, 0.015625]...
Solution: [ 0.03043061  0.         -0.00111708  0.00376453 -0.0003996 ]...
Iterations: 9999
Final Gap: 0.00027067967329087316
Indices of significant gap changes: [ 1 15 27 35 42 47 50 53 56 57 58 59 60 61 63 64]
CPU time at significant changes: [0.       0.109375 0.21875  0.3125   0.375    0.421875 0.421875 0.4375
 0.46875  0.46875  0.46875  0.484375 0.484375 0.484375 0.484375 0.484375]


In [38]:
import numpy as np
import time
from math import sqrt

def soft_thresholding(x, lambda_):
    """Apply soft thresholding for LASSO."""
    return np.sign(x) * np.maximum(np.abs(x) - lambda_, 0.0)

def lasso_objective(X, y, beta, lambda_):
    """Compute LASSO objective function."""
    n = len(y)
    residual = y - X.dot(beta)
    loss = 0.5 * np.dot(residual, residual) / n
    penalty = lambda_ * np.linalg.norm(beta, ord=1)
    return loss + penalty

def lasso_gradient(X, y, beta, lambda_):
    """Compute gradient for LASSO."""
    n = len(y)
    residual = y - X.dot(beta)
    gradient_loss = -X.T.dot(residual) / n
    gradient_penalty = lambda_ * np.sign(beta)
    return gradient_loss + gradient_penalty

def find_magnitude_indices(gap_history, func=np.log2):
    """Identify indices where the log magnitude of the gap changes significantly."""
    magnitudes = np.floor(func(gap_history))
    unique_magnitudes = []
    indices = []
    for i, magnitude in enumerate(magnitudes):
        if magnitude not in unique_magnitudes:
            unique_magnitudes.append(magnitude)
            indices.append(i + 1)  # Storing 1-based index for clarity
    indices = np.array(indices)
    return indices, unique_magnitudes

class AcceleratedLassoMethod:
    def __init__(self, X, y, lambda_, lr=0.01, max_iter=1000, tol=1e-6):
        self.X = X
        self.y = y
        self.lambda_ = lambda_
        self.lr = lr
        self.max_iter = max_iter
        self.tol = tol
        self.beta = np.zeros(X.shape[1])
        self.t = 1
        self.loss_history = []
        self.gap_history = []
        self.cpu_time_history = []

    def compute_steps(self):
        beta_prev = np.zeros_like(self.beta)

        for i in range(self.max_iter):
            start_time = time.process_time()
            t_prev = self.t
            self.t = 0.5 * (1 + sqrt(1 + 4 * t_prev**2))
            y_tilde = self.beta + ((t_prev - 1) / self.t) * (self.beta - beta_prev)
            gradient = lasso_gradient(self.X, self.y, y_tilde, self.lambda_)
            beta_prev = self.beta.copy()
            self.beta = soft_thresholding(y_tilde - self.lr * gradient, self.lr * self.lambda_)
            loss = lasso_objective(self.X, self.y, self.beta, self.lambda_)
            self.loss_history.append(loss)
            end_time = time.process_time()
            self.cpu_time_history.append(end_time - start_time)

            # Compute convergence gap and check termination condition
            gap = np.linalg.norm(self.beta - beta_prev) / (np.linalg.norm(beta_prev) if np.linalg.norm(beta_prev) != 0 else 1)
            self.gap_history.append(gap)
            if gap < self.tol:
                break

        final_gap = self.gap_history[-1] if self.gap_history else None
        iterations = i + 1
        indices_of_changes, _ = find_magnitude_indices(self.gap_history)
        cpu_times_at_changes = np.cumsum(self.cpu_time_history)[indices_of_changes - 1]  # Adjust for 0-based index

        return self.beta, iterations, final_gap, indices_of_changes, cpu_times_at_changes


lambda_ = 0.5
solver = AcceleratedLassoMethod(A, b, lambda_, max_iter=10000, tol=1e-12)
solution, iterations, final_gap, indices, cpu_time = solver.compute_steps()

# Output results for analysis
print(f"Solution: {solution[:5]}...")  # Showing first few elements of the solution
print(f"Iterations: {iterations}")
print(f"Final Gap: {final_gap}")
print(f"Indices of significant gap changes: {indices}")
print(f"CPU time at significant changes: {cpu_time}")


Solution: [ 0.  0.  0. -0.  0.]...
Iterations: 10000
Final Gap: 0.044637258385165045
Indices of significant gap changes: [ 1  2  3  4 10 28]
CPU time at significant changes: [0.015625 0.015625 0.046875 0.046875 0.078125 0.171875]


In [79]:
import numpy as np

def generate_sparse_least_squares(m, n, rho):
    A = np.random.uniform(-1, 1, (m, n))
    x_star = np.zeros(n)
    non_zero_indices = np.random.choice(n, int(n * rho), replace=False)
    x_star[non_zero_indices] = np.random.normal(0, 1, int(n * rho))
    b = np.dot(A, x_star)  # Assuming no noise for simplicity
    return A, b, x_star

def grad_f(x, A, b):
    return A.T @ (A @ x - b)

def nag_for_least_squares(A, b, x0, L0, max_iter=1000, epsilon=1e-6):
    x_k = x0
    y_k = x0
    a_k = 0
    L = L0
    A_k = 0
    initial_gap = np.linalg.norm(grad_f(x0, A, b))
    
    def T_L(y, L):
        return y - grad_f(y, A, b) / L
    
    gap_history = []

    for k in range(max_iter):
        a = (1 + np.sqrt(1 + 4 * L * A_k)) / (2 * L)
        A_next = A_k + a
        
        y = (A_k * x_k + a * y_k) / A_next
        T_L_y = T_L(y, L)
        
        while np.linalg.norm(grad_f(T_L_y, A, b) - grad_f(y, A, b)) > L * np.linalg.norm(T_L_y - y):
            L *= 2  # Adjust L
            T_L_y = T_L(y, L)
        
        x_next = T_L(y, L)
        y_k = x_next
        x_k = x_next
        A_k = A_next
        
        current_gap = np.linalg.norm(grad_f(x_k, A, b))
        relative_gap = current_gap / initial_gap
        gap_history.append(relative_gap)
        
        if current_gap < epsilon:
            break
    
    return x_k, gap_history

# Example usage
m, n, rho = 1000, 4000, 0.1
A, b, x_star = generate_sparse_least_squares(m, n, rho)
x0 = np.zeros(n)  # Initial point
L0 = 1.0          # Initial Lipschitz constant guess

solution_nag, gap_history = nag_for_least_squares(A, b, x0, L0)
print("NAG Solution (first 5 elements):", solution_nag[::])
print("True Solution (first 5 elements):", x_star[::])
print("Relative Gap History:", gap_history)


NAG Solution (first 5 elements): [ 0.11190684 -0.35738204 -0.18666485 ... -0.02723769 -0.09114664
 -0.28201096]
True Solution (first 5 elements): [0. 0. 0. ... 0. 0. 0.]
Relative Gap History: [0.5345670844678049, 0.3301030680457549, 0.2259003333232547, 0.16543333448267675, 0.1267499759139684, 0.10021281181678422, 0.08106500704992278, 0.06671877830177501, 0.055654506613736054, 0.046924853248181, 0.03991034454104063, 0.034189683035946196, 0.02946673736206275, 0.025527420345436758, 0.022213180429108467, 0.019404134251082453, 0.01700800142361202, 0.014952643859415264, 0.013180909226067603, 0.011646986232023244, 0.010313776047255205, 0.009150962007484097, 0.008133569047566397, 0.007240873042508832, 0.006455564409291675, 0.005763099306197503, 0.005151191158436198, 0.00460940845518985, 0.0041288539276763325, 0.003701906675829155, 0.003322013430317374, 0.00298351848610559, 0.002681524303261869, 0.002411776597558518, 0.0021705691145686547, 0.001954664319942114, 0.001761227032675436, 0.001587768